In [ ]:
%matplotlib inline
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from models import GaussianDistribution, build
from hmc_sampler import hamiltonian
from sklearn import preprocessing
from surrogate import NeuralGrad

In [ ]:
# Dimension of parameter space and number of training points
dim_gaussian = 20
num_training = 1000

In [ ]:
# Gaussian distribution with mean and  covariance Matrix
mu = np.zeros(dim_gaussian)
var = np.ones(dim_gaussian)
test_model = GaussianDistribution(mu,var)

In [ ]:
# Generate training data

training_data = test_model.sample(num_training)
#print(training_data.shape)
energy = np.zeros(training_data.shape[0])
gradient = np.zeros((training_data.shape[0],training_data.shape[1]))
for i in range(training_data.shape[0]):
    energy[i] = test_model.energy(training_data[i,:])
    gradient[i] =test_model.gradient(training_data[i,:])


In [ ]:
fig = plt.figure(figsize=(6,4))
plt.xlim(-5,5)
plt.ylim(-5,5)
plt.plot(training_data[:,0],training_data[:,1],'o',alpha=1.5)
plt.show()


In [ ]:
 # Train the NN to approximate gradient

scaled_data = preprocessing.StandardScaler().fit(training_data)
training_data = preprocessing.scale(training_data)
model = build(dim_gaussian, [50], [dim_gaussian])
model.fit(training_data, gradient, epochs=10, batch_size=20)

In [ ]:
 #Using the NN inside HMC to sample points
grad_hat = NeuralGrad(model,scaled_data)
nnghmc_draws = hamiltonian(model=test_model, sample_size=1000, leapfrog_steps=10, stepsize=0.1,surrogate=grad_hat)[0]

In [ ]:
# Trace plots to see convergence
fig = plt.figure(figsize=(10, 2))
plt.plot(nnghmc_draws[:, 0])


In [ ]:
#NNgHMC draws 
fig = plt.figure(figsize=(6, 4), dpi=200)
plt.xlim(-5, 5)
plt.ylim(-5, 5)
plt.plot(nnghmc_draws[:, 0], nnghmc_draws[:, 1], 'o', alpha=0.5,label='NNgHMC')
plt.plot(training_data[:,0],training_data[:,1],'ro',alpha=0.5,label="training")
plt.legend()
# plt.savefig("result.jpg")